In [2]:
import pandas as pd
import numpy as np
import nltk
import sklearn as sk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import platform 
import torch
import sys

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-13.2.1-arm64-arm-64bit
PyTorch Version: 2.0.0.dev20230212

Python 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
[Clang 14.0.6 ]
Pandas 1.5.3
Scikit-Learn 1.2.1
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [4]:
# df = pd.read_csv('text_data_nathan.csv', usecols = ['Utterance', 'Label'])
# df.head()

In [13]:
train_df = pd.read_csv('train_features_audio_aligned__with_wav_files__embeddings.csv', usecols = ['Utterance', 'Label'])
train_df.head()

,Utterance,Label
0,otherwise theyre pretty smelly,sv
1,she got the treatments,sd
2,this is the reality,^q
3,so we kind of looked around,sd
4,hes in in florida jail now,sd


In [14]:
train_df.shape

(6305, 2)

In [15]:
test_df = pd.read_csv('test_features_audio_aligned__with_wav_files__embeddings.csv', usecols = ['Utterance', 'Label'])
test_df.head()

,Utterance,Label
0,and one is uh four,sd
1,regarding uh taxes i you know,"fo_o_fw_""_by_bc"
2,and that was actually after the war was over,sd
3,oh well he made it at fifty,sd
4,and another thing is cost,sv


In [16]:
test_df = pd.read_csv('test_features_audio_aligned__with_wav_files__embeddings.csv', usecols = ['Utterance', 'Label'])
test_df.head()

,Utterance,Label
0,and one is uh four,sd
1,regarding uh taxes i you know,"fo_o_fw_""_by_bc"
2,and that was actually after the war was over,sd
3,oh well he made it at fifty,sd
4,and another thing is cost,sv


In [17]:
train_df.shape

(6305, 2)

In [46]:
test_df.shape

(333, 2)

In [45]:
test_df = test_df[test_df['Label'] != '^g']

In [19]:
# df['Utterance'] = df['Utterance'].str.lower()
# df.head()

In [20]:
train_df['Utterance'] = train_df['Utterance'].str.lower()
train_df.head()

,Utterance,Label
0,otherwise theyre pretty smelly,sv
1,she got the treatments,sd
2,this is the reality,^q
3,so we kind of looked around,sd
4,hes in in florida jail now,sd


In [21]:
test_df['Utterance'] = test_df['Utterance'].str.lower()
test_df.head()

,Utterance,Label
0,and one is uh four,sd
1,regarding uh taxes i you know,"fo_o_fw_""_by_bc"
2,and that was actually after the war was over,sd
3,oh well he made it at fifty,sd
4,and another thing is cost,sv


In [22]:
def remove_html(x):
    return BeautifulSoup(x,'html.parser').get_text(separator=" ")

In [23]:
# df['Utterance'] = df['Utterance'].apply(lambda x: remove_html(x))
# df.head()

In [24]:
train_df['Utterance'] = train_df['Utterance'].apply(lambda x: remove_html(x))
train_df.head()

,Utterance,Label
0,otherwise theyre pretty smelly,sv
1,she got the treatments,sd
2,this is the reality,^q
3,so we kind of looked around,sd
4,hes in in florida jail now,sd


In [25]:
test_df['Utterance'] = test_df['Utterance'].apply(lambda x: remove_html(x))
test_df.head()

,Utterance,Label
0,and one is uh four,sd
1,regarding uh taxes i you know,"fo_o_fw_""_by_bc"
2,and that was actually after the war was over,sd
3,oh well he made it at fifty,sd
4,and another thing is cost,sv


In [26]:
# df['Utterance'] = df['Utterance'].str\
#                     .replace('[^a-zA-Z ]', '',regex=True)
# df['Utterance'] = df['Utterance'].str\
#                     .replace('http\S+|www.\S+', '',regex=True)
# df.head()

In [27]:
train_df['Utterance'] = train_df['Utterance'].str\
                    .replace('[^a-zA-Z ]', '',regex=True)
train_df['Utterance'] = train_df['Utterance'].str\
                    .replace('http\S+|www.\S+', '',regex=True)
train_df.head()

,Utterance,Label
0,otherwise theyre pretty smelly,sv
1,she got the treatments,sd
2,this is the reality,^q
3,so we kind of looked around,sd
4,hes in in florida jail now,sd


In [28]:
test_df['Utterance'] = test_df['Utterance'].str\
                    .replace('[^a-zA-Z ]', '',regex=True)
test_df['Utterance'] = test_df['Utterance'].str\
                    .replace('http\S+|www.\S+', '',regex=True)
test_df.head()

,Utterance,Label
0,and one is uh four,sd
1,regarding uh taxes i you know,"fo_o_fw_""_by_bc"
2,and that was actually after the war was over,sd
3,oh well he made it at fifty,sd
4,and another thing is cost,sv


In [29]:
# df['Utterance'] = df['Utterance'].apply(lambda x:contractions.fix(x))
# df.head()

In [30]:
train_df['Utterance'] = train_df['Utterance'].apply(lambda x:contractions.fix(x))
train_df.head()

,Utterance,Label
0,otherwise they are pretty smelly,sv
1,she got the treatments,sd
2,this is the reality,^q
3,so we kind of looked around,sd
4,hes in in florida jail now,sd


In [31]:
test_df['Utterance'] = test_df['Utterance'].apply(lambda x:contractions.fix(x))
test_df.head()

,Utterance,Label
0,and one is uh four,sd
1,regarding uh taxes i you know,"fo_o_fw_""_by_bc"
2,and that was actually after the war was over,sd
3,oh well he made it at fifty,sd
4,and another thing is cost,sv


In [47]:
test_df['Label'].value_counts()

sd                 161
sv                  57
%                   19
qy                  18
qw                  10
ba                   9
b                    6
^q                   5
na                   5
aa                   5
ad                   4
bf                   4
fc                   3
qy^d                 3
ng                   3
h                    3
fo_o_fw_"_by_bc      2
qo                   2
^h                   2
bk                   2
bh                   2
^2                   1
fp                   1
qh                   1
fa                   1
b^m                  1
no                   1
ny                   1
t1                   1
Name: Label, dtype: int64

In [48]:
x_train = train_df['Utterance']
y_train = train_df['Label']

In [49]:
x_test = test_df['Utterance']
y_test = test_df['Label']

In [35]:
# from sklearn.model_selection import train_test_split
# from sklearn.utils import shuffle

# shuffled_df = shuffle(df)

# x_train, x_test, y_train, y_test = train_test_split(
#                     shuffled_df['Utterance'], shuffled_df['Label'], 
#                     train_size = 0.95, test_size = 0.05, shuffle=True)

In [13]:
del df, shuffled_df

In [50]:
import torch
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")
from tqdm import tqdm
def extract_bert(list_of_sentences=[], batch_size=32):
    cls_hidden_states = []
    if len(list_of_sentences)<batch_size: #go longer ones we should batch
        tknzed= tokenizer(list_of_sentences, return_tensors="pt",padding=True)
        b=model(**tknzed)
        cls_hidden_states = torch.squeeze(b.last_hidden_state[:,0,:])
    else:
        for i in tqdm(range(0,len(list_of_sentences),batch_size)):    
            tknzed= tokenizer(list_of_sentences[i:min(len(list_of_sentences),i+batch_size)], return_tensors="pt",padding=True)
            b=model(**tknzed)
            cls_hidden_states.append(torch.squeeze(b.last_hidden_state[:,0,:]))
    return cls_hidden_states

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
# x_train_bert = extract_bert(list(x_train))
# x_train_torch = torch.cat(x_train_bert)
# x_test_bert = extract_bert(list(x_test))
# x_test_torch = torch.cat(x_test_bert)

In [21]:
for i in range(0, len(x_train), 1000):
    reduced_x_train = x_train[i: i + 1000]
    reduced_x_train_bert = extract_bert(list(reduced_x_train))
    reduced_x_train_torch = torch.cat(reduced_x_train_bert)
    redcued_x_train_features = reduced_x_train_torch.cpu().detach().numpy()
    print(i)
    if i != 0:
        x_train_features = np.concatenate((x_train_features, redcued_x_train_features), axis = 0)
    else:
        x_train_features = redcued_x_train_features
#     reduced_y_train = np.asarray(reduced_y_train)
    print(x_train_features.shape)
    del reduced_x_train, reduced_x_train_bert, reduced_x_train_torch, redcued_x_train_features

100%|███████████████████████████████████████████| 32/32 [00:05<00:00,  5.88it/s]


0
(1000, 768)


100%|███████████████████████████████████████████| 32/32 [00:05<00:00,  5.63it/s]


1000
(2000, 768)


100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  6.82it/s]


...

4000
(5000, 768)


100%|███████████████████████████████████████████| 32/32 [00:05<00:00,  5.52it/s]


5000
(6000, 768)


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.24it/s]

6000
(6305, 768)


In [22]:
# reduced_x_train_bert = extract_bert(list(x_train[189000:-1]))
# print(len(reduced_x_train_bert[0]))
# reduced_x_train_torch = torch.cat(reduced_x_train_bert)
# redcued_x_train_features = reduced_x_train_torch.cpu().detach().numpy()

In [23]:
# x_train_features = np.concatenate((x_train_features, redcued_x_train_features), axis = 0)

In [24]:
# del reduced_x_train, reduced_x_train_bert, reduced_x_train_torch, redcued_x_train_features

In [25]:
x_train_features.shape

(6305, 768)

In [26]:
x_train.shape

(6305,)

In [52]:
for i in range(0, len(x_test), 1000):
    reduced_x_test = x_test[i: i + 1000]
    reduced_x_test_bert = extract_bert(list(reduced_x_test))
    reduced_x_test_torch = torch.cat(reduced_x_test_bert)
    redcued_x_test_features = reduced_x_test_torch.cpu().detach().numpy()
    print(i)
    if i != 0:
        x_test_features = np.concatenate((x_test_features, redcued_x_test_features), axis = 0)
    else:
        x_test_features = redcued_x_test_features
#     reduced_y_train = np.asarray(reduced_y_train)
    print(x_test_features.shape)
    del reduced_x_test, reduced_x_test_bert, reduced_x_test_torch, redcued_x_test_features

100%|███████████████████████████████████████████| 11/11 [00:01<00:00,  6.21it/s]


0
(333, 768)


In [28]:
x_train_features.shape

(6305, 768)

In [53]:
x_test_features.shape

(333, 768)

In [30]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [31]:
y_train.shape

(6305,)

In [54]:
y_test.shape

(333,)

In [33]:
with open('features/train_features_roberta_audio_aligned.npy', 'wb') as f:
    np.save(f, x_train_features)
    
with open('features/train_labels_roberta_audio_aligned.npy', 'wb') as f:
    np.save(f, y_train)

In [55]:
with open('features/test_features_roberta_audio_aligned.npy', 'wb') as f:
    np.save(f, x_test_features)
    
with open('features/test_labels_roberta_audio_aligned.npy', 'wb') as f:
    np.save(f, y_test)

In [77]:
x_train = np.load('features/train_features_roberta.npy', allow_pickle=True)
y_train = np.load('features/train_labels_roberta.npy', allow_pickle=True)

x_test = np.load('features/test_features_roberta.npy', allow_pickle=True)
y_test = np.load('features/test_labels_roberta.npy', allow_pickle=True)

In [80]:
x_test_audio = np.load('features/test_features_roberta_audio_aligned.npy', allow_pickle=True)
y_test_audio = np.load('features/test_labels_roberta_audio_aligned.npy', allow_pickle=True)

In [81]:
x_test_audio.shape

(333, 768)

In [82]:
y_test_audio.shape

(333,)

In [83]:
x_train.shape

(189753, 768)

In [84]:
y_train.shape

(189753,)

In [85]:
x_test.shape

(9987, 768)

In [86]:
y_test.shape

(9987,)

In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(x_train)
simple_x_train = scaler.transform(x_train)

simple_x_test =scaler.transform(x_test)

In [87]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_train)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

{'%': 0, '^2': 1, '^g': 2, '^h': 3, '^q': 4, 'aa': 5, 'aap_am': 6, 'ad': 7, 'ar': 8, 'arp_nd': 9, 'b': 10, 'b^m': 11, 'ba': 12, 'bd': 13, 'bf': 14, 'bh': 15, 'bk': 16, 'br': 17, 'fa': 18, 'fc': 19, 'fo_o_fw_"_by_bc': 20, 'fp': 21, 'ft': 22, 'h': 23, 'na': 24, 'ng': 25, 'nn': 26, 'no': 27, 'ny': 28, 'oo_co_cc': 29, 'qh': 30, 'qo': 31, 'qrr': 32, 'qw': 33, 'qw^d': 34, 'qy': 35, 'qy^d': 36, 'sd': 37, 'sv': 38, 't1': 39, 't3': 40}


In [88]:
y_test_audio = le.transform(y_test_audio)

In [89]:
y_train.shape

(189753,)

In [33]:
model = Perceptron()

model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.593463,0.754545,0.664380,770.000000
1,0.000000,0.000000,0.000000,41.000000
2,0.000000,0.000000,0.000000,1.000000
3,0.270833,0.419355,0.329114,31.000000
4,0.035714,0.024390,0.028986,41.000000
5,0.606061,0.172117,0.268097,581.000000
6,0.000000,0.000000,0.000000,7.000000
7,0.393939,0.302326,0.342105,43.000000
8,0.000000,0.000000,0.000000,14.000000
9,0.000000,0.000000,0.000000,9.000000


In [34]:
from imblearn.pipeline import Pipeline as imbpipeline


# pipeline = imbpipeline(steps = [['classifier', LinearSVC()]])

# param_grid = {'classifier__C': [0.01, 0.1, 1, 10, 100, 1000],
#               'classifier__penalty': ['l1', 'l2'],
#               'classifier__loss': ['hinge', 'squared_hinge'],
#              'classifier__class_weight': [None]}


svc = LinearSVC()

svc.fit(x_train, y_train)
pred = svc.predict(x_test)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.768212,0.753247,0.760656,770.000000
1,0.333333,0.024390,0.045455,41.000000
2,0.000000,0.000000,0.000000,1.000000
3,0.588235,0.322581,0.416667,31.000000
4,0.000000,0.000000,0.000000,41.000000
5,0.548476,0.340792,0.420382,581.000000
6,0.000000,0.000000,0.000000,7.000000
7,0.777778,0.325581,0.459016,43.000000
8,0.000000,0.000000,0.000000,14.000000
9,0.000000,0.000000,0.000000,9.000000


In [35]:
# from imblearn.pipeline import Pipeline as imbpipeline


# pipeline = imbpipeline(steps = [['classifier', LogisticRegression()]])

# param_grid = {'classifier__C': np.logspace(-4, 4, 50),
#               'classifier__penalty': ['l1', 'l2'],
#              'classifier__class_weight': [None]}

lg = LogisticRegression()
# lg = GridSearchCV(estimator=pipeline,
#                            param_grid=param_grid,
#                            scoring='f1_weighted',
#                            cv=10,
#                            n_jobs=-1,
#                             verbose = 1,
#                             refit = True)

lg.fit(x_train, y_train)
pred = lg.predict(x_test)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.440499,0.596104,0.506623,770.000000
1,0.000000,0.000000,0.000000,41.000000
2,0.000000,0.000000,0.000000,1.000000
3,0.000000,0.000000,0.000000,31.000000
4,0.000000,0.000000,0.000000,41.000000
5,0.302158,0.144578,0.195576,581.000000
6,0.000000,0.000000,0.000000,7.000000
7,0.000000,0.000000,0.000000,43.000000
8,0.000000,0.000000,0.000000,14.000000
9,0.000000,0.000000,0.000000,9.000000


In [10]:
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression()
# clf.fit(x_train, y_train)
# clf.score(x_test, y_test)

In [11]:
# train_y_pred = clf.predict(x_train)

# train_result = classification_report(train_y_pred, y_train)
# print(train_result)

# y_pred = clf.predict(x_test)

# result = classification_report(y_pred, y_test)
# print(result)

In [47]:
# model = MultinomialNB()
# model.fit(x_train, y_train)
# pred = model.predict(x_test)
# pred = classification_report(y_test, pred, output_dict=True)
# report = pd.DataFrame(pred).transpose()
# report

In [46]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.865854,0.554688,0.676190,128.000000
1,0.000000,0.000000,0.000000,7.000000
2,0.000000,0.000000,0.000000,1.000000
3,1.000000,0.166667,0.285714,6.000000
4,0.000000,0.000000,0.000000,10.000000
5,0.769231,0.396040,0.522876,101.000000
6,1.000000,1.000000,1.000000,1.000000
7,1.000000,0.272727,0.428571,11.000000
8,0.000000,0.000000,0.000000,4.000000
9,0.000000,0.000000,0.000000,3.000000


In [ ]:
# model = LinearSVC()

# model.fit(x_train, y_train)
# pred = model.predict(x_test)
# pred = classification_report(y_test, pred)
# print(pred)

In [48]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

,precision,recall,f1-score,support
0,0.852632,0.632812,0.726457,128.000000
1,0.000000,0.000000,0.000000,7.000000
2,0.000000,0.000000,0.000000,1.000000
3,1.000000,0.166667,0.285714,6.000000
4,0.000000,0.000000,0.000000,10.000000
5,0.769231,0.396040,0.522876,101.000000
6,1.000000,1.000000,1.000000,1.000000
7,0.555556,0.454545,0.500000,11.000000
8,0.000000,0.000000,0.000000,4.000000
9,0.000000,0.000000,0.000000,3.000000


In [90]:
x_train_torch = torch.from_numpy(x_train)
x_test_torch = torch.from_numpy(x_test)
x_test_audio_torch = torch.from_numpy(x_test_audio)

In [91]:
x_test_audio_torch.shape

torch.Size([333, 768])

In [92]:
y_test_audio.shape

(333,)

In [93]:
zipper = lambda x,y : list(zip(x,list(y)))

In [94]:
train_torch = zipper(x_train_torch, y_train)
test_torch = zipper(x_test_torch, y_test)
test_audio_torch = zipper(x_test_audio_torch, y_test_audio)

train_loader = torch.utils.data.DataLoader(train_torch, batch_size=256)
test_loader = torch.utils.data.DataLoader(test_torch, batch_size=256)
test_audio_loader = torch.utils.data.DataLoader(test_audio_torch, batch_size=256)

In [95]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    
    def __init__(self):
        super(Net, self).__init__() 
#         self.gru = nn.GRU(input_size=768, hidden_size=32, batch_first=True) 
        self.fc1 = nn.Linear(768,41)
#         self.fc2 = nn.Linear(256, 64)
#         self.fc3 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

#         self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
#         out, hn = self.gru(x, torch.randn(1, len(x), 32))
#         out = self.fc1(self.relu(x))
#         out = self.fc2(self.relu(out))
        out = self.fc1(x)
        return out

In [96]:
linear_model = Net()
print(linear_model)

Net(
  (fc1): Linear(in_features=768, out_features=41, bias=True)
  (relu): ReLU()
)


In [97]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(linear_model.parameters(), lr=0.05)

In [98]:
# number of epochs to train the model
n_epochs = 2500

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    linear_model.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = linear_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    linear_model.eval() # prep model for evaluation
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = linear_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}' \
          .format(epoch+1, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...'.format(valid_loss_min, valid_loss))
        
        torch.save(linear_model.state_dict(), 'bert_model_linear.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 2.051567 	Validation Loss: 1.986048
Validation loss decreased (inf --> 1.986048).         Saving model ...
Epoch: 2 	Training Loss: 1.913626 	Validation Loss: 1.879416
Validation loss decreased (1.986048 --> 1.879416).         Saving model ...
Epoch: 3 	Training Loss: 1.827345 	Validation Loss: 1.806525
Validation loss decreased (1.879416 --> 1.806525).         Saving model ...
Epoch: 4 	Training Loss: 1.765834 	Validation Loss: 1.752761
Validation loss decreased (1.806525 --> 1.752761).         Saving model ...
Epoch: 5 	Training Loss: 1.718950 	Validation Loss: 1.710723
Validation loss decreased (1.752761 --> 1.710723).         Saving model ...
Epoch: 6 	Training Loss: 1.681424 	Validation Loss: 1.676437
Validation loss decreased (1.710723 --> 1.676437).         Saving model ...
Epoch: 7 	Training Loss: 1.650290 	Validation Loss: 1.647591
Validation loss decreased (1.676437 --> 1.647591).         Saving model ...
Epoch: 8 	Training Loss: 1.623753 	Validation 

Epoch: 61 	Training Loss: 1.244667 	Validation Loss: 1.252452
Validation loss decreased (1.255337 --> 1.252452).         Saving model ...
Epoch: 62 	Training Loss: 1.241819 	Validation Loss: 1.249620
Validation loss decreased (1.252452 --> 1.249620).         Saving model ...
Epoch: 63 	Training Loss: 1.239025 	Validation Loss: 1.246841
Validation loss decreased (1.249620 --> 1.246841).         Saving model ...
Epoch: 64 	Training Loss: 1.236282 	Validation Loss: 1.244113
Validation loss decreased (1.246841 --> 1.244113).         Saving model ...
Epoch: 65 	Training Loss: 1.233589 	Validation Loss: 1.241435
Validation loss decreased (1.244113 --> 1.241435).         Saving model ...
Epoch: 66 	Training Loss: 1.230945 	Validation Loss: 1.238804
Validation loss decreased (1.241435 --> 1.238804).         Saving model ...
Epoch: 67 	Training Loss: 1.228348 	Validation Loss: 1.236221
Validation loss decreased (1.238804 --> 1.236221).         Saving model ...
Epoch: 68 	Training Loss: 1.225797

Epoch: 121 	Training Loss: 1.133683 	Validation Loss: 1.141842
Validation loss decreased (1.143054 --> 1.141842).         Saving model ...
Epoch: 122 	Training Loss: 1.132483 	Validation Loss: 1.140644
Validation loss decreased (1.141842 --> 1.140644).         Saving model ...
Epoch: 123 	Training Loss: 1.131296 	Validation Loss: 1.139460
Validation loss decreased (1.140644 --> 1.139460).         Saving model ...
Epoch: 124 	Training Loss: 1.130123 	Validation Loss: 1.138288
Validation loss decreased (1.139460 --> 1.138288).         Saving model ...
Epoch: 125 	Training Loss: 1.128962 	Validation Loss: 1.137129
Validation loss decreased (1.138288 --> 1.137129).         Saving model ...
Epoch: 126 	Training Loss: 1.127814 	Validation Loss: 1.135983
Validation loss decreased (1.137129 --> 1.135983).         Saving model ...
Epoch: 127 	Training Loss: 1.126679 	Validation Loss: 1.134849
Validation loss decreased (1.135983 --> 1.134849).         Saving model ...
Epoch: 128 	Training Loss: 

Epoch: 180 	Training Loss: 1.079813 	Validation Loss: 1.088077
Validation loss decreased (1.088775 --> 1.088077).         Saving model ...
Epoch: 181 	Training Loss: 1.079119 	Validation Loss: 1.087384
Validation loss decreased (1.088077 --> 1.087384).         Saving model ...
Epoch: 182 	Training Loss: 1.078429 	Validation Loss: 1.086697
Validation loss decreased (1.087384 --> 1.086697).         Saving model ...
Epoch: 183 	Training Loss: 1.077745 	Validation Loss: 1.086016
Validation loss decreased (1.086697 --> 1.086016).         Saving model ...
Epoch: 184 	Training Loss: 1.077066 	Validation Loss: 1.085339
Validation loss decreased (1.086016 --> 1.085339).         Saving model ...
Epoch: 185 	Training Loss: 1.076392 	Validation Loss: 1.084667
Validation loss decreased (1.085339 --> 1.084667).         Saving model ...
Epoch: 186 	Training Loss: 1.075724 	Validation Loss: 1.084001
Validation loss decreased (1.084667 --> 1.084001).         Saving model ...
Epoch: 187 	Training Loss: 

Epoch: 239 	Training Loss: 1.046096 	Validation Loss: 1.054526
Validation loss decreased (1.054995 --> 1.054526).         Saving model ...
Epoch: 240 	Training Loss: 1.045626 	Validation Loss: 1.054059
Validation loss decreased (1.054526 --> 1.054059).         Saving model ...
Epoch: 241 	Training Loss: 1.045158 	Validation Loss: 1.053595
Validation loss decreased (1.054059 --> 1.053595).         Saving model ...
Epoch: 242 	Training Loss: 1.044694 	Validation Loss: 1.053133
Validation loss decreased (1.053595 --> 1.053133).         Saving model ...
Epoch: 243 	Training Loss: 1.044231 	Validation Loss: 1.052675
Validation loss decreased (1.053133 --> 1.052675).         Saving model ...
Epoch: 244 	Training Loss: 1.043772 	Validation Loss: 1.052218
Validation loss decreased (1.052675 --> 1.052218).         Saving model ...
Epoch: 245 	Training Loss: 1.043314 	Validation Loss: 1.051765
Validation loss decreased (1.052218 --> 1.051765).         Saving model ...
Epoch: 246 	Training Loss: 

...

Epoch: 2245 	Training Loss: 0.868725 	Validation Loss: 0.883696
Validation loss decreased (0.883717 --> 0.883696).         Saving model ...
Epoch: 2246 	Training Loss: 0.868702 	Validation Loss: 0.883675
Validation loss decreased (0.883696 --> 0.883675).         Saving model ...
Epoch: 2247 	Training Loss: 0.868679 	Validation Loss: 0.883654
Validation loss decreased (0.883675 --> 0.883654).         Saving model ...
Epoch: 2248 	Training Loss: 0.868656 	Validation Loss: 0.883634
Validation loss decreased (0.883654 --> 0.883634).         Saving model ...
Epoch: 2249 	Training Loss: 0.868633 	Validation Loss: 0.883613
Validation loss decreased (0.883634 --> 0.883613).         Saving model ...
Epoch: 2250 	Training Loss: 0.868610 	Validation Loss: 0.883592
Validation loss decreased (0.883613 --> 0.883592).         Saving model ...
Epoch: 2251 	Training Loss: 0.868588 	Validation Loss: 0.883571
Validation loss decreased (0.883592 --> 0.883571).         Saving model ...
Epoch: 2252 	Trainin

Epoch: 2304 	Training Loss: 0.867396 	Validation Loss: 0.882489
Validation loss decreased (0.882509 --> 0.882489).         Saving model ...
Epoch: 2305 	Training Loss: 0.867373 	Validation Loss: 0.882469
Validation loss decreased (0.882489 --> 0.882469).         Saving model ...
Epoch: 2306 	Training Loss: 0.867351 	Validation Loss: 0.882449
Validation loss decreased (0.882469 --> 0.882449).         Saving model ...
Epoch: 2307 	Training Loss: 0.867329 	Validation Loss: 0.882428
Validation loss decreased (0.882449 --> 0.882428).         Saving model ...
Epoch: 2308 	Training Loss: 0.867307 	Validation Loss: 0.882408
Validation loss decreased (0.882428 --> 0.882408).         Saving model ...
Epoch: 2309 	Training Loss: 0.867285 	Validation Loss: 0.882388
Validation loss decreased (0.882408 --> 0.882388).         Saving model ...
Epoch: 2310 	Training Loss: 0.867263 	Validation Loss: 0.882368
Validation loss decreased (0.882388 --> 0.882368).         Saving model ...
Epoch: 2311 	Trainin

Epoch: 2363 	Training Loss: 0.866112 	Validation Loss: 0.881326
Validation loss decreased (0.881346 --> 0.881326).         Saving model ...
Epoch: 2364 	Training Loss: 0.866091 	Validation Loss: 0.881307
Validation loss decreased (0.881326 --> 0.881307).         Saving model ...
Epoch: 2365 	Training Loss: 0.866070 	Validation Loss: 0.881288
Validation loss decreased (0.881307 --> 0.881288).         Saving model ...
Epoch: 2366 	Training Loss: 0.866048 	Validation Loss: 0.881268
Validation loss decreased (0.881288 --> 0.881268).         Saving model ...
Epoch: 2367 	Training Loss: 0.866027 	Validation Loss: 0.881249
Validation loss decreased (0.881268 --> 0.881249).         Saving model ...
Epoch: 2368 	Training Loss: 0.866006 	Validation Loss: 0.881230
Validation loss decreased (0.881249 --> 0.881230).         Saving model ...
Epoch: 2369 	Training Loss: 0.865984 	Validation Loss: 0.881210
Validation loss decreased (0.881230 --> 0.881210).         Saving model ...
Epoch: 2370 	Trainin

Epoch: 2422 	Training Loss: 0.864873 	Validation Loss: 0.880206
Validation loss decreased (0.880225 --> 0.880206).         Saving model ...
Epoch: 2423 	Training Loss: 0.864852 	Validation Loss: 0.880188
Validation loss decreased (0.880206 --> 0.880188).         Saving model ...
Epoch: 2424 	Training Loss: 0.864831 	Validation Loss: 0.880169
Validation loss decreased (0.880188 --> 0.880169).         Saving model ...
Epoch: 2425 	Training Loss: 0.864811 	Validation Loss: 0.880151
Validation loss decreased (0.880169 --> 0.880151).         Saving model ...
Epoch: 2426 	Training Loss: 0.864790 	Validation Loss: 0.880132
Validation loss decreased (0.880151 --> 0.880132).         Saving model ...
Epoch: 2427 	Training Loss: 0.864770 	Validation Loss: 0.880113
Validation loss decreased (0.880132 --> 0.880113).         Saving model ...
Epoch: 2428 	Training Loss: 0.864749 	Validation Loss: 0.880095
Validation loss decreased (0.880113 --> 0.880095).         Saving model ...
Epoch: 2429 	Trainin

Epoch: 2481 	Training Loss: 0.863674 	Validation Loss: 0.879127
Validation loss decreased (0.879145 --> 0.879127).         Saving model ...
Epoch: 2482 	Training Loss: 0.863654 	Validation Loss: 0.879109
Validation loss decreased (0.879127 --> 0.879109).         Saving model ...
Epoch: 2483 	Training Loss: 0.863634 	Validation Loss: 0.879091
Validation loss decreased (0.879109 --> 0.879091).         Saving model ...
Epoch: 2484 	Training Loss: 0.863614 	Validation Loss: 0.879073
Validation loss decreased (0.879091 --> 0.879073).         Saving model ...
Epoch: 2485 	Training Loss: 0.863594 	Validation Loss: 0.879055
Validation loss decreased (0.879073 --> 0.879055).         Saving model ...
Epoch: 2486 	Training Loss: 0.863574 	Validation Loss: 0.879037
Validation loss decreased (0.879055 --> 0.879037).         Saving model ...
Epoch: 2487 	Training Loss: 0.863555 	Validation Loss: 0.879019
Validation loss decreased (0.879037 --> 0.879019).         Saving model ...
Epoch: 2488 	Trainin

In [99]:
linear_model = Net()
linear_model.load_state_dict(torch.load("bert_model_linear.pt"))
linear_model.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)

for data, target in train_loader:
    output = linear_model(data)
    _, preds = torch.max(output, 1)
    train_pred_list = torch.cat([train_pred_list, preds.view(-1)])
    train_target_list = torch.cat([train_target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             train_count+=1
            
# print("Training Accuracy =", train_count / len(train_loader.dataset))
train_result = classification_report(train_pred_list.numpy(), 
                                     train_target_list.numpy())
print("Training Classification report: \n", train_result)
            
for data, target in test_loader:
    output = linear_model(data)
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy())
print("Testing Classification report: \n", result)

Training Classification report: 
               precision    recall  f1-score   support

           0       0.71      0.77      0.74     13205
           1       0.02      0.30      0.04        47
           2       0.00      0.00      0.00         0
           3       0.37      0.77      0.50       251
           4       0.01      0.54      0.01        13
           5       0.33      0.50      0.40      6994
           6       0.00      0.00      0.00         0
           7       0.22      0.55      0.31       274
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.95      0.74      0.83     46896
          11       0.01      0.23      0.01        22
          12       0.62      0.73      0.67      3795
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         1
          15       0.38      0.65      0.48       580
          16       0.31      0.60      0.40    

In [100]:
linear_model = Net()
linear_model.load_state_dict(torch.load("bert_model_linear.pt"))
linear_model.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0
output_proba = False

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)
            
for data, target in test_audio_loader:
    output = linear_model(data)
    output_np = output.cpu().detach().numpy()
    if np.any(output_proba):
        output_proba = np.concatenate((output_proba, output_np), axis = 0)
    else:
        output_proba = output_np
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy())
print("Testing Classification report: \n", result)

Testing Classification report: 
               precision    recall  f1-score   support

           0       0.47      0.47      0.47        19
           1       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.40      0.40      0.40         5
           7       0.25      0.50      0.33         2
          10       0.67      0.67      0.67         6
          11       0.00      0.00      0.00         0
          12       0.67      0.67      0.67         9
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         1
          16       0.50      0.50      0.50         2
          18       0.00      0.00      0.00         0
          19       0.67      0.67      0.67         3
          20       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
          23       0.33      1.00      0.50     

In [101]:
output_proba.shape

(333, 41)

In [102]:
with open('test_audio_probabilities.npy', 'wb') as f:
    np.save(f, output_proba)

In [103]:
output_proba = np.load('test_audio_probabilities.npy', allow_pickle = True)
output_proba.shape

(333, 41)

In [44]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    
    def __init__(self):
        super(Net, self).__init__() 
#         self.gru = nn.GRU(input_size=768, hidden_size=32, batch_first=True) 
        self.fc1 = nn.Linear(768,256)
        self.fc2 = nn.Linear(256, 42)
#         self.fc3 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

#         self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
#         out, hn = self.gru(x, torch.randn(1, len(x), 32))
#         out = self.fc1(self.relu(x))
#         out = self.fc2(self.relu(out))
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        return out

In [45]:
two_layer = Net()
print(two_layer)

Net(
  (fc1): Linear(in_features=768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=42, bias=True)
  (relu): ReLU()
)


In [46]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(two_layer.parameters(), lr=0.01)

In [47]:
# number of epochs to train the model
n_epochs = 500

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    two_layer.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = two_layer(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    two_layer.eval() # prep model for evaluation
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = two_layer(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}' \
          .format(epoch+1, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...'.format(valid_loss_min, valid_loss))
        
        torch.save(two_layer.state_dict(), 'bert_two_layer.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 2.169632 	Validation Loss: 2.102942
Validation loss decreased (inf --> 2.102942).         Saving model ...
Epoch: 2 	Training Loss: 2.101434 	Validation Loss: 2.081255
Validation loss decreased (2.102942 --> 2.081255).         Saving model ...
Epoch: 3 	Training Loss: 2.068668 	Validation Loss: 2.036421
Validation loss decreased (2.081255 --> 2.036421).         Saving model ...
Epoch: 4 	Training Loss: 2.003332 	Validation Loss: 1.957762
Validation loss decreased (2.036421 --> 1.957762).         Saving model ...
Epoch: 5 	Training Loss: 1.906732 	Validation Loss: 1.865701
Validation loss decreased (1.957762 --> 1.865701).         Saving model ...
Epoch: 6 	Training Loss: 1.812231 	Validation Loss: 1.785919
Validation loss decreased (1.865701 --> 1.785919).         Saving model ...
Epoch: 7 	Training Loss: 1.747003 	Validation Loss: 1.703382
Validation loss decreased (1.785919 --> 1.703382).         Saving model ...
Epoch: 8 	Training Loss: 1.704493 	Validation 

Epoch: 61 	Training Loss: 1.252388 	Validation Loss: 1.284673
Validation loss decreased (1.289096 --> 1.284673).         Saving model ...
Epoch: 62 	Training Loss: 1.247852 	Validation Loss: 1.280270
Validation loss decreased (1.284673 --> 1.280270).         Saving model ...
Epoch: 63 	Training Loss: 1.243405 	Validation Loss: 1.275612
Validation loss decreased (1.280270 --> 1.275612).         Saving model ...
Epoch: 64 	Training Loss: 1.239090 	Validation Loss: 1.271332
Validation loss decreased (1.275612 --> 1.271332).         Saving model ...
Epoch: 65 	Training Loss: 1.234912 	Validation Loss: 1.266129
Validation loss decreased (1.271332 --> 1.266129).         Saving model ...
Epoch: 66 	Training Loss: 1.230897 	Validation Loss: 1.262105
Validation loss decreased (1.266129 --> 1.262105).         Saving model ...
Epoch: 67 	Training Loss: 1.226922 	Validation Loss: 1.258300
Validation loss decreased (1.262105 --> 1.258300).         Saving model ...
Epoch: 68 	Training Loss: 1.223114

Epoch: 121 	Training Loss: 1.101171 	Validation Loss: 1.127989
Validation loss decreased (1.129076 --> 1.127989).         Saving model ...
Epoch: 122 	Training Loss: 1.099502 	Validation Loss: 1.126873
Validation loss decreased (1.127989 --> 1.126873).         Saving model ...
Epoch: 123 	Training Loss: 1.097910 	Validation Loss: 1.125727
Validation loss decreased (1.126873 --> 1.125727).         Saving model ...
Epoch: 124 	Training Loss: 1.096300 	Validation Loss: 1.124161
Validation loss decreased (1.125727 --> 1.124161).         Saving model ...
Epoch: 125 	Training Loss: 1.094714 	Validation Loss: 1.123057
Validation loss decreased (1.124161 --> 1.123057).         Saving model ...
Epoch: 126 	Training Loss: 1.093171 	Validation Loss: 1.121704
Validation loss decreased (1.123057 --> 1.121704).         Saving model ...
Epoch: 127 	Training Loss: 1.091585 	Validation Loss: 1.120941
Validation loss decreased (1.121704 --> 1.120941).         Saving model ...
Epoch: 128 	Training Loss: 

Epoch: 182 	Training Loss: 1.025512 	Validation Loss: 1.083740
Epoch: 183 	Training Loss: 1.024594 	Validation Loss: 1.083029
Validation loss decreased (1.083608 --> 1.083029).         Saving model ...
Epoch: 184 	Training Loss: 1.023620 	Validation Loss: 1.082657
Validation loss decreased (1.083029 --> 1.082657).         Saving model ...
Epoch: 185 	Training Loss: 1.022731 	Validation Loss: 1.082520
Validation loss decreased (1.082657 --> 1.082520).         Saving model ...
Epoch: 186 	Training Loss: 1.021809 	Validation Loss: 1.082034
Validation loss decreased (1.082520 --> 1.082034).         Saving model ...
Epoch: 187 	Training Loss: 1.020928 	Validation Loss: 1.081575
Validation loss decreased (1.082034 --> 1.081575).         Saving model ...
Epoch: 188 	Training Loss: 1.020010 	Validation Loss: 1.081063
Validation loss decreased (1.081575 --> 1.081063).         Saving model ...
Epoch: 189 	Training Loss: 1.019119 	Validation Loss: 1.080473
Validation loss decreased (1.081063 --> 

Epoch: 249 	Training Loss: 0.977077 	Validation Loss: 1.050219
Validation loss decreased (1.050942 --> 1.050219).         Saving model ...
Epoch: 250 	Training Loss: 0.976520 	Validation Loss: 1.052821
Epoch: 251 	Training Loss: 0.976012 	Validation Loss: 1.048938
Validation loss decreased (1.050219 --> 1.048938).         Saving model ...
Epoch: 252 	Training Loss: 0.975466 	Validation Loss: 1.048927
Validation loss decreased (1.048938 --> 1.048927).         Saving model ...
Epoch: 253 	Training Loss: 0.974938 	Validation Loss: 1.047253
Validation loss decreased (1.048927 --> 1.047253).         Saving model ...
Epoch: 254 	Training Loss: 0.974398 	Validation Loss: 1.046868
Validation loss decreased (1.047253 --> 1.046868).         Saving model ...
Epoch: 255 	Training Loss: 0.973899 	Validation Loss: 1.046378
Validation loss decreased (1.046868 --> 1.046378).         Saving model ...
Epoch: 256 	Training Loss: 0.973361 	Validation Loss: 1.047119
Epoch: 257 	Training Loss: 0.972809 	Val

Epoch: 328 	Training Loss: 0.944535 	Validation Loss: 1.018997
Validation loss decreased (1.020599 --> 1.018997).         Saving model ...
Epoch: 329 	Training Loss: 0.944214 	Validation Loss: 1.019978
Epoch: 330 	Training Loss: 0.943912 	Validation Loss: 1.017997
Validation loss decreased (1.018997 --> 1.017997).         Saving model ...
Epoch: 331 	Training Loss: 0.943616 	Validation Loss: 1.017617
Validation loss decreased (1.017997 --> 1.017617).         Saving model ...
Epoch: 332 	Training Loss: 0.943278 	Validation Loss: 1.018820
Epoch: 333 	Training Loss: 0.942975 	Validation Loss: 1.019032
Epoch: 334 	Training Loss: 0.942656 	Validation Loss: 1.018585
Epoch: 335 	Training Loss: 0.942350 	Validation Loss: 1.018344
Epoch: 336 	Training Loss: 0.942054 	Validation Loss: 1.016613
Validation loss decreased (1.017617 --> 1.016613).         Saving model ...
Epoch: 337 	Training Loss: 0.941755 	Validation Loss: 1.016201
Validation loss decreased (1.016613 --> 1.016201).         Saving 

Epoch: 402 	Training Loss: 0.924901 	Validation Loss: 1.000904
Epoch: 403 	Training Loss: 0.924623 	Validation Loss: 0.999993
Validation loss decreased (1.000883 --> 0.999993).         Saving model ...
Epoch: 404 	Training Loss: 0.924415 	Validation Loss: 1.000293
Epoch: 405 	Training Loss: 0.924188 	Validation Loss: 0.999895
Validation loss decreased (0.999993 --> 0.999895).         Saving model ...
Epoch: 406 	Training Loss: 0.923981 	Validation Loss: 0.999646
Validation loss decreased (0.999895 --> 0.999646).         Saving model ...
Epoch: 407 	Training Loss: 0.923751 	Validation Loss: 1.000289
Epoch: 408 	Training Loss: 0.923530 	Validation Loss: 0.998656
Validation loss decreased (0.999646 --> 0.998656).         Saving model ...
Epoch: 409 	Training Loss: 0.923305 	Validation Loss: 0.999694
Epoch: 410 	Training Loss: 0.923107 	Validation Loss: 0.998850
Epoch: 411 	Training Loss: 0.922886 	Validation Loss: 1.000427
Epoch: 412 	Training Loss: 0.922660 	Validation Loss: 0.998447
Val

Epoch: 485 	Training Loss: 0.908606 	Validation Loss: 0.982352
Epoch: 486 	Training Loss: 0.908430 	Validation Loss: 0.981490
Epoch: 487 	Training Loss: 0.908242 	Validation Loss: 0.981305
Epoch: 488 	Training Loss: 0.908099 	Validation Loss: 0.979771
Validation loss decreased (0.981032 --> 0.979771).         Saving model ...
Epoch: 489 	Training Loss: 0.907909 	Validation Loss: 0.981084
Epoch: 490 	Training Loss: 0.907745 	Validation Loss: 0.980075
Epoch: 491 	Training Loss: 0.907555 	Validation Loss: 0.979977
Epoch: 492 	Training Loss: 0.907392 	Validation Loss: 0.979470
Validation loss decreased (0.979771 --> 0.979470).         Saving model ...
Epoch: 493 	Training Loss: 0.907234 	Validation Loss: 0.979512
Epoch: 494 	Training Loss: 0.907087 	Validation Loss: 0.979547
Epoch: 495 	Training Loss: 0.906916 	Validation Loss: 0.979761
Epoch: 496 	Training Loss: 0.906728 	Validation Loss: 0.978475
Validation loss decreased (0.979470 --> 0.978475).         Saving model ...
Epoch: 497 	Trai

In [48]:
two_layer = Net()
two_layer.load_state_dict(torch.load("bert_two_layer.pt"))
two_layer.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)

for data, target in train_loader:
    output = two_layer(data)
    _, preds = torch.max(output, 1)
    train_pred_list = torch.cat([train_pred_list, preds.view(-1)])
    train_target_list = torch.cat([train_target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             train_count+=1
            
# print("Training Accuracy =", train_count / len(train_loader.dataset))
train_result = classification_report(train_pred_list.numpy(), 
                                     train_target_list.numpy())
print("Training Classification report: \n", train_result)
            
for data, target in test_loader:
    output = two_layer(data)
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy())
print("Testing Classification report: \n", result)

Training Classification report: 
               precision    recall  f1-score   support

           0       0.55      0.90      0.68      8761
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.09      0.96      0.16        49
           4       0.00      0.00      0.00         0
           5       0.43      0.33      0.38     13634
           6       0.00      0.00      0.00         0
           7       0.00      0.29      0.01         7
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.96      0.73      0.83     47884
          11       0.00      0.00      0.00         0
          12       0.52      0.80      0.63      2903
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.35      0.60      0.44       581
          16       0.00      0.12      0.00    

In [49]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    
    def __init__(self):
        super(Net, self).__init__() 
#         self.gru = nn.GRU(input_size=768, hidden_size=32, batch_first=True) 
        self.fc1 = nn.Linear(768,384)
        self.fc2 = nn.Linear(384, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 42)
        self.relu = nn.ReLU()

#         self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
#         out, hn = self.gru(x, torch.randn(1, len(x), 32))
        out = self.fc1(self.relu(x))
        out = self.fc2(self.relu(out))
        out = self.fc3(self.relu(out))
        out = self.fc4(x)
        return out

In [50]:
three_layer = Net()
print(three_layer)

Net(
  (fc1): Linear(in_features=768, out_features=384, bias=True)
  (fc2): Linear(in_features=384, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=42, bias=True)
  (relu): ReLU()
)


In [51]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(three_layer.parameters(), lr=0.03)

In [52]:
# number of epochs to train the model
n_epochs = 150

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    three_layer.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = three_layer(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    three_layer.eval() # prep model for evaluation
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = three_layer(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}' \
          .format(epoch+1, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...'.format(valid_loss_min, valid_loss))
        
        torch.save(three_layer.state_dict(), 'bert_three_layer.pt')
        valid_loss_min = valid_loss

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x768 and 128x42)

In [ ]:
three_layer = Net()
three_layer.load_state_dict(torch.load("bert_three_layer.pt"))
three_layer.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)

for data, target in train_loader:
    output = three_layer(data)
    _, preds = torch.max(output, 1)
    train_pred_list = torch.cat([train_pred_list, preds.view(-1)])
    train_target_list = torch.cat([train_target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             train_count+=1
            
# print("Training Accuracy =", train_count / len(train_loader.dataset))
train_result = classification_report(train_pred_list.numpy(), 
                                     train_target_list.numpy())
print("Training Classification report: \n", train_result)
            
for data, target in test_loader:
    output = three_layer(data)
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy(), output_dict = True)
result = pd.DataFrame(result).transpose()
print("Testing Classification report: \n", result)